# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [97]:
%matplotlib inline
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp
import matplotlib.pyplot as plt

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [98]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')

Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [99]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)


In [100]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,11,25,,72,,2017-02-16 02:24:13 PM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,39
1,,22,9,,79,,2017-02-16 02:25:19 PM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,18,8,,82,,2017-02-16 02:24:10 PM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,1,In Service,False,27
3,,26,34,,83,,2017-02-16 02:24:16 PM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,,8,31,,116,,2017-02-16 02:24:05 PM,40.741776,,-74.001497,,W 17 St & 8 Ave,,W 17 St & 8 Ave,1,In Service,False,39


In [101]:
# there is one row for each bike station.  How many stations are there? 
len(df)

666

In [102]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s = gp.GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

(-74.096936600000006,
 40.661063371900603,
 -73.929891100000006,
 40.804212999999997)

In [103]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

(957370.14732175611,
 180120.27032614074,
 1003695.8507545569,
 232275.23054640222)

In [104]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

367 stations in Manhattan


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [105]:
def bike_to_boro(x):
    boro = boros.geometry[x]
    in_boro = df.geometry.within(boro)
    return print(sum(in_boro), 'stations in ', boros.loc[x,'BoroName'])

In [106]:
#This function calls for the number of bikes in each boro
for i in range(0,5):
    bike_to_boro(i)

0 stations in  Staten Island
236 stations in  Brooklyn
13 stations in  Queens
367 stations in  Manhattan
0 stations in  Bronx


In [120]:
def bike_in_boro(x):
    boro = boros.geometry[x]
    in_boro = df.geometry.within(boro)
    return in_boro

In [125]:
for j in range (0,5):
    b = bike_in_boro(j)    
    if j == 0 and b == True:
        df('boro') = 0
    elif j == 1 and b == True:
        df('boro') = 1
    elif j == 2 and b == True:
        df('boro') = 2
    elif j == 3 and b == True:
        df('boro') = 3
    elif j == 4 and b == True:
        df('boro') = 4
    else:
        df('boro') = NaN

SyntaxError: can't assign to function call (<ipython-input-125-635063686413>, line 12)